## Process

In [57]:
import os,time, random
from multiprocessing import Process, Pool

In [38]:
# win 无法使用
# os.fork()
print(os.getpid())
print(os.getppid())

1324
15604


In [46]:
def run_child(name):
    print('running child process, %s (%s)'% (name, os.getpid()))
print('parent process %s'% os.getpid())
p = Process(target=run_child, args='test')
p.start()
# p.join()


parent process 1324


In [52]:
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

print('Parent process %s.' % os.getpid())
p = Process(target=run_proc, args=('test',))
print('Process will start.')
p.start()
p.join()
print('Process end.')

Parent process 1324.
Process will start.
Process end.


In [ ]:
def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))
print('Parent process %s.' % os.getpid())
# 默认是 pool(4), 也可以用 p=pool(5), 于是同时跑5个 process,
# 如果你不幸拥有 8 核 CPU，你要提交至少 9 个子进程才能看到上面的等待效果
p = Pool()
for i in range(5):
    p.apply_async(long_time_task, args=(i,))
print('Waiting for all subprocesses done...')
p.close()
p.join()
print('All subprocesses done.')

Parent process 1324.
Waiting for all subprocesses done...


## Thread

Python 的线程是真正的 Posix Thread，而不是模拟出来的线程. threading是高级模块,是对低级模块thread的封装.


In [4]:
import time, threading, multiprocessing

In [18]:
# 启动一个线程就是把一个函数传入并创建 Thread 实例，然后调用 start()开始执行
def test_loop():
    print('thread %s is running'% threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print('thread %s is running >>> %s'% (threading.current_thread().name, n))
        time.sleep(0.3)
    print('thread %s ended'% threading.current_thread().name)
print('thread %s is running'% threading.current_thread().name)
t = threading.Thread(target=test_loop, name='name-test')
t.start()
t.join()
print('thread %s ended'% threading.current_thread().name)


thread MainThread is running
thread name-test is running
thread name-test is running >>> 1
thread name-test is running >>> 2
thread name-test is running >>> 3
thread name-test is running >>> 4
thread name-test is running >>> 5
thread name-test ended
thread MainThread ended


In [26]:
#  current_thread()函数，它永远返回当前线程的实例
threading.current_thread?
threading.current_thread()

<_MainThread(MainThread, started 8564)>

Signature: threading.current_thread()
Docstring:
Return the current Thread object, corresponding to the caller's thread of control.

If the caller's thread of control was not created through the threading
module, a dummy thread object with limited functionality is returned.
File:      d:\software\anaconda3\envs\ai\lib\threading.py
Type:      function


多线程和多进程最大的不同在于，多进程中，同一个变量，各自有一份拷贝存在于每个进
程中，互不影响，而多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以
被任何一个线程修改，因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，
把内容给改乱了。

In [3]:
balance = 0
lock = threading.Lock()
def change_it(n):
    global balance
    balance += n
    balance -= n
def thread_run(n):
    for i in range(1000000):
        lock.acquire()
        try:
            change_it(n)
        finally:
            lock.release()
            pass# 如果不释放锁，就相当于死锁
t1 = threading.Thread(target=thread_run, args=(5,))
t2 = threading.Thread(target=thread_run, args=(17,))
t1.start()
t2.start()
t1.join()
t2.join()
balance

0

因为 Python 的线程虽然是真正的线程，但解释器执行代码时，有一个 GIL 锁：Global
Interpreter Lock，任何 Python 线程执行前，必须先获得 GIL 锁，然后，每执行 100 条字
节码，解释器就自动释放 GIL 锁，让别的线程有机会执行。这个 GIL 全局锁实际上把所有线
程的执行代码都给上了锁，所以，多线程在 Python 中只能交替执行，即使 100 个线程跑
在 100 核 CPU 上，也只能用到 1 个核。

GIL 是 Python **解释器设计的历史遗留问题**，通常我们用的解释器是官方实现的 CPython，
要真正利用多核，除非重写一个不带 GIL 的解释器。

所以，在 Python 中，可以使用多线程，**但不要指望能有效利用多核**。如果一定要通过多线
程利用多核，那只能通过 C 扩展来实现，不过这样就失去了 Python 简单易用的特点。

不过，也不用过于担心，Python 虽然不能利用多线程实现多核任务，但可以通过多进程实
现多核任务。多个 Python 进程有各自独立的 GIL 锁，互不影响。

In [3]:
def dead_loop():
    x = 0
    while True:
        x = x * 1.0
for i in range(multiprocessing.cpu_count()):
    t = threading.Thread(target=dead_loop)
    t.start()

NameError: name 'multiprocessing' is not defined

### Coroutine(协程)

协程，又称微线程，纤程。协程的概念很早就提出来了，但直到最近几年才在某些语言（如 Lua）中得到广泛应用。

子程序，或者称为函数，在所有语言中都是层级调用，比如 A 调用 B，B 在执行过程中又调用了 C，C 执行完毕返回，B 执行完毕返回，最后是 A 执行完毕。所以子程序调用是通过栈实现的，一个线程就是执行一个子程序。

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同。协程看上去也是子程序，但执行过程中，在子程序内部可中断，然后转而执行别的子程序，在适当的时候再返回来接着执行。

In [24]:
import time

def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        time.sleep(3)
        r = '200 OK'
def produce(c):
    next(c)
    n = 0
    while n < 4:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

In [25]:
c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK


In [33]:
c = consumer()
next(c)
n = 1
print('[PRODUCER] Producing %s...' % n)
r = c.send(n)
print('[PRODUCER] Consumer return: %s' % n)

n = 2
print('[PRODUCER] Producing %s...' % n)
r = c.send(n)
print('[PRODUCER] Consumer return: %s' % n)


[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 1
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 2


Python 通过 yield 提供了对协程的基本支持，但是不完全。而第三方的 gevent 为 Python 提供了比较完善的协程支持。

使用 gevent，可以获得极高的并发性能，但 gevent 只能在 Unix/Linux 下运行，在 Windows 下不保证正常安装和运行。

In [35]:
import gevent

ModuleNotFoundError: No module named 'gevent'